In [1]:
import os
from pathlib import Path
from pprint import pprint
import re
import shutil
from uuid import uuid4
from itertools import chain

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

%cd /root/nerve/python
import nerve.tools as tools
from nerve.parser import AssetNameParser
from nerve.database import Database
from nerve.database_test import DatabaseTests
from nerve.specification_base import *
from nerve.specifications import *
from nerve.validators import *
from nerve.traits import *

/root/nerve/python


In [3]:
root = '/tmp/projects'
if Path(root).exists():
    shutil.rmtree(root)
os.makedirs(root)
fullpaths = DatabaseTests().create_files(root)

In [4]:
!tree /tmp/projects

/tmp/projects
├── proj001
│   ├── spec001
│   │   └── pizza
│   │       ├── p-proj001_s-spec001_d-pizza_v001
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v001_c000-001_f0002.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v001_c000-001_f0003.png
│   │       ├── p-proj001_s-spec001_d-pizza_v002
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0003.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-000_f0004.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0001.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0002.png
│   │       │   ├── p-proj001_s-spec001_d-pizza_v002_c000-001_f0003.png
│   │       │   └── p-proj001_s-spec001_d-pizza_v002_c000-001_f0004.png
│   │       └── p-proj001_s-spec001_d-pizza_v003
│

In [4]:
class Spec001(SequenceSpecificationBase):
    name = 'spec001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'coordinate', 'frame', 'extension']
    coordinate = IntType(required=True, validators=[is_coordinate])

class Spec002(SequenceSpecificationBase):
    name = 'spec002'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'frame', 'extension']
    width = IntType(required=True, validators=[lambda x: is_equal(x, 1024)])
    height = IntType(required=True, validators=[lambda x: is_equal(x, 1024)])
    extension = StringType(required=True, validators=[is_extension, lambda x: is_equal(x, 'exr')])
    
class Vdb001(FileSpecificationBase):
    name = 'vdb001'
    filename_fields = ['project', 'specification', 'descriptor', 'version', 'extension']
    extension = StringType(required=True, validators=[is_extension, lambda x: is_equal(x, 'vdb')])

# db = Database(root, [Spec001, Spec002, Vdb001])
db = Database(root, [Spec001, Spec002])
db.update()
data = db.data
data.head()

,project,specification,descriptor,version,coordinate,frame,extension,filename,fullpath,error,asset_name,asset_path,asset_type
0,proj001,spec001,pizza,1.0,"[0, 1]",1.0,png,p-proj001_s-spec001_d-pizza_v001_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,NaN,p-proj001_s-spec001_d-pizza_v001,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
1,proj001,spec001,pizza,1.0,"[0, 1]",2.0,png,p-proj001_s-spec001_d-pizza_v001_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,NaN,p-proj001_s-spec001_d-pizza_v001,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
2,proj001,spec001,pizza,1.0,"[0, 1]",3.0,png,p-proj001_s-spec001_d-pizza_v001_c000-001_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,NaN,p-proj001_s-spec001_d-pizza_v001,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
3,proj001,spec001,pizza,2.0,"[0, 0]",1.0,png,p-proj001_s-spec001_d-pizza_v002_c000-000_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,NaN,p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence
4,proj001,spec001,pizza,2.0,"[0, 0]",2.0,png,p-proj001_s-spec001_d-pizza_v002_c000-000_f000...,/tmp/projects/proj001/spec001/pizza/p-proj001_...,NaN,p-proj001_s-spec001_d-pizza_v002,/tmp/projects/proj001/spec001/pizza/p-proj001_...,sequence


In [5]:
data.groupby('asset_path', as_index=False).error.agg(lambda x: x.tolist()).loc[2, 'error']
x = data.groupby('asset_path', as_index=False).agg(lambda x: x.tolist()).apply(lambda x: x.to_dict(), axis=1)
x.tolist()[2]['error']
x[0]
x

0    {'asset_path': '/tmp/projects/proj001/spec001/...
1    {'asset_path': '/tmp/projects/proj001/spec001/...
2    {'asset_path': '/tmp/projects/proj001/spec001/...
3    {'asset_path': '/tmp/projects/proj001/spec002/...
4    {'asset_path': '/tmp/projects/proj001/spec002/...
dtype: object